# Introduction to CUDA C++

In [1]:
!nvidia-smi

Fri Jul 25 19:10:29 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   33C    P0             47W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

## Example 1

In [2]:
%%writefile hello_gpu.cu
// File: hello_gpu.cu
#include <stdio.h>

/*
 * Kernel: A function that runs on the device (GPU).
 * The __global__ specifier marks it as such.
 * This kernel will be executed by many threads in parallel.
 */
__global__ void hello_from_gpu()
{
    printf("Hello, World! from the GPU!\n");
}

/*
 * Host: The main function that runs on the CPU.
 * It orchestrates the program and launches kernels on the device.
 */
int main()
{
    // 1. A message from the host CPU.
    printf("Hello from the host CPU before launching the kernel.\n");

    // 2. The Host launches the kernel on the Device.
    // We launch a "grid" of 1 block, containing 4 threads.
    hello_from_gpu<<<1, 4>>>();

    // 3. The host must wait for the device to finish its work before exiting.
    // cudaDeviceSynchronize() is a barrier that pauses the host
    // until all previously launched device tasks are complete.
    cudaDeviceSynchronize();

    // 4. A final message from the host CPU.
    printf("Kernel launch finished. Hello from the host CPU again.\n");

    return 0;
}


Writing hello_gpu.cu


In [3]:
!nvcc hello_gpu.cu -o hello_gpu \
    -gencode arch=compute_75,code=sm_75 \
    -gencode arch=compute_80,code=sm_80 \
    -gencode arch=compute_80,code=compute_80

In [4]:
!./hello_gpu

Hello from the host CPU before launching the kernel.
Hello, World! from the GPU!
Hello, World! from the GPU!
Hello, World! from the GPU!
Hello, World! from the GPU!
Kernel launch finished. Hello from the host CPU again.


## Example 2

In [5]:
%%writefile demo_indices.cu
// File: demo_indices.cu
#include <stdio.h>
#include <cuda_runtime.h>

/*
 * Kernel to demonstrate thread indexing.
 * Each thread calculates its global index, and the first thread
 * in each block prints its identity to show the hierarchy.
 */
__global__ void demoIndices(int N)
{

    // 1. Calculate the unique global index for this thread.
    int idx = blockIdx.x * blockDim.x + threadIdx.x;

    // 2. Bounds check to ensure we don't do work for padded threads.
    if (idx < N)
    {
        // 3. To keep output clean, we will only have the FIRST thread
        // of each block print its information. This confirms that
        // multiple blocks are being launched and are aware of their IDs.
        if (threadIdx.x == 0)
        {
            printf("GPU: Block ID = %d, Thread ID = %d -> Calculated Global Index = %d\n",
                   blockIdx.x, threadIdx.x, idx);
        }
    }
}

/*
 * Host: The main function that runs on the CPU.
 */
int main()
{
    // Define the size of our conceptual data.
    // We'll use a small N that is not a perfect multiple of THREADS_PER_BLOCK
    // to show that the bounds check and grid calculation work correctly.
    int N = 500;

    // Define the number of threads per block. 256 is a common, efficient choice.
    int THREADS_PER_BLOCK = 256;

    // Calculate the number of blocks needed in the grid, using the
    // integer arithmetic trick for ceiling division.
    // For N=500 and Threads=256, this will be ceil(500/256) = 2 blocks.
    int BLOCKS_PER_GRID = (N + THREADS_PER_BLOCK - 1) / THREADS_PER_BLOCK;

    printf("Host: Problem size N = %d\n", N);
    printf("Host: Threads per Block = %d\n", THREADS_PER_BLOCK);
    printf("Host: Calculated Blocks in Grid = %d\n\n", BLOCKS_PER_GRID);

    // Launch the kernel with our calculated grid and block dimensions.
    demoIndices<<<BLOCKS_PER_GRID, THREADS_PER_BLOCK>>>(N);

    // Synchronize to make sure the kernel finishes and we see its output.
    cudaDeviceSynchronize();

    printf("\nHost: Kernel finished.\n");

    return 0;
}


Writing demo_indices.cu


In [6]:
!nvcc demo_indices.cu -o demo_indices \
    -gencode arch=compute_75,code=sm_75 \
    -gencode arch=compute_80,code=sm_80 \
    -gencode arch=compute_80,code=compute_80

In [7]:
!./demo_indices

Host: Problem size N = 500
Host: Threads per Block = 256
Host: Calculated Blocks in Grid = 2

GPU: Block ID = 0, Thread ID = 0 -> Calculated Global Index = 0
GPU: Block ID = 1, Thread ID = 0 -> Calculated Global Index = 256

Host: Kernel finished.


## Example 3

In [8]:
%%writefile vector_add.cu
// File: vector_add.cu
#include <stdio.h>
#include <stdlib.h>  // For malloc/free
#include <stdbool.h> // For bool type

// The vectorAdd kernel from the previous section.
__global__ void vectorAdd(const float *d_A, const float *d_B, float *d_C, int N)
{
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < N)
    {
        d_C[idx] = d_A[idx] + d_B[idx];
    }
}

/*
 * Host: The main function that runs on the CPU.
 */
int main()
{
    // --- 1. Host-side Setup ---
    int N = 1024 * 1024;
    size_t size = N * sizeof(float);

    // Allocate host memory for vectors A, B, and C
    float *h_A = (float *)malloc(size);
    float *h_B = (float *)malloc(size);
    float *h_C = (float *)malloc(size);

    // Initialize host vectors
    for (int i = 0; i < N; ++i)
    {
        h_A[i] = 1.0f;
        h_B[i] = 2.0f;
    }

    // --- 2. Device-side Memory Allocation ---
    // Declare device pointers
    float *d_A, *d_B, *d_C;
    // Allocate memory on the GPU for each vector
    cudaMalloc(&d_A, size);
    cudaMalloc(&d_B, size);
    cudaMalloc(&d_C, size);

    // --- 3. Copy Input Data from Host to Device ---
    printf("Copying input data from Host to Device...\n");
    cudaMemcpy(d_A, h_A, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_B, h_B, size, cudaMemcpyHostToDevice);

    // --- 4. Launch the Kernel ---
    int THREADS_PER_BLOCK = 256;
    int BLOCKS_PER_GRID = (N + THREADS_PER_BLOCK - 1) / THREADS_PER_BLOCK;

    printf("Launching vectorAdd kernel...\n");
    vectorAdd<<<BLOCKS_PER_GRID, THREADS_PER_BLOCK>>>(d_A, d_B, d_C, N);

    // Block host execution until the device kernel finishes
    cudaDeviceSynchronize();
    printf("Kernel execution finished.\n");

    // --- 5. Copy Result Data from Device to Host ---
    printf("Copying result data from Device to Host...\n");
    cudaMemcpy(h_C, d_C, size, cudaMemcpyDeviceToHost);

    // --- 6. Verification (on the host) ---
    bool success = true;
    // Check all elements for correctness
    for (int i = 0; i < N; ++i)
    {
        if (h_C[i] != 3.0f)
        {
            printf("Error at index %d: Expected 3.0, got %f\n", i, h_C[i]);
            success = false;
            break;
        }
    }
    if (success)
    {
        // Print one of the results to show it worked
        printf("Verification successful! e.g., h_C[100] = %f\n", h_C[100]);
    }

    // --- 7. Cleanup ---
    // Free device memory
    cudaFree(d_A);
    cudaFree(d_B);
    cudaFree(d_C);
    // Free host memory
    free(h_A);
    free(h_B);
    free(h_C);

    return 0;
}


Writing vector_add.cu


In [9]:
!nvcc vector_add.cu -o vector_add \
    -gencode arch=compute_75,code=sm_75 \
    -gencode arch=compute_80,code=sm_80 \
    -gencode arch=compute_80,code=compute_80

In [10]:
!./vector_add

Copying input data from Host to Device...
Launching vectorAdd kernel...
Kernel execution finished.
Copying result data from Device to Host...
Verification successful! e.g., h_C[100] = 3.000000


## Example 4

In [11]:
%%writefile random_generation.cu
// File: random_generation.cu
#include <stdio.h>
#include <stdlib.h>
#include <time.h>          // For time()
#include <curand_kernel.h> // Required for cuRAND device functions

/*
 * Kernel 1: Initializes the cuRAND state for each thread.
 * Each thread gets a unique state based on its ID and a seed.
 */
__global__ void setup_kernel(curandState_t *states, unsigned long seed)
{
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    // Initialize the generator state for this thread
    curand_init(seed,          // The seed for the generator
                idx,           // A unique sequence number for each thread
                0,             // A 0 offset
                &states[idx]); // The address of the state to initialize
}

/*
 * Kernel 2: Uses the initialized states to generate random numbers.
 */
__global__ void generate_kernel(float *output, curandState_t *states, int N)
{
    int idx = blockIdx.x * blockDim.x + threadIdx.x;

    if (idx < N)
    {
        // Copy the state from global memory to a register for this thread
        curandState_t localState = states[idx];

        // Generate a random float between 0.0 and 1.0
        output[idx] = curand_uniform(&localState);

        // Copy the updated state back to global memory
        states[idx] = localState;
    }
}

// The main host function
int main()
{
    int N = 1024;
    size_t size = N * sizeof(float);

    // --- 1. Host-side Setup ---
    float *h_output = (float *)malloc(size);

    // --- 2. Device-side Memory Allocation ---
    float *d_output;
    curandState_t *d_states;
    cudaMalloc(&d_output, size);
    cudaMalloc(&d_states, N * sizeof(curandState_t));

    // --- 3. Kernel Configuration ---
    int THREADS_PER_BLOCK = 256;
    int BLOCKS_PER_GRID = (N + THREADS_PER_BLOCK - 1) / THREADS_PER_BLOCK;

    // --- 4. Launch Setup Kernel ---
    // Use the current time as a seed for the random number generator
    setup_kernel<<<BLOCKS_PER_GRID, THREADS_PER_BLOCK>>>(d_states, time(NULL));

    // --- 5. Launch Generation Kernel ---
    generate_kernel<<<BLOCKS_PER_GRID, THREADS_PER_BLOCK>>>(d_output, d_states, N);
    cudaDeviceSynchronize();

    // --- 6. Copy results back to host ---
    cudaMemcpy(h_output, d_output, size, cudaMemcpyDeviceToHost);

    // --- 7. Verification ---
    printf("Generated random numbers (first 10):\n");
    for (int i = 0; i < 10; ++i)
    {
        printf("h_output[%d] = %f\n", i, h_output[i]);
    }

    // --- 8. Cleanup ---
    cudaFree(d_states);
    cudaFree(d_output);
    free(h_output);

    return 0;
}


Writing random_generation.cu


In [12]:
!nvcc random_generation.cu -o random_generation \
    -gencode arch=compute_75,code=sm_75 \
    -gencode arch=compute_80,code=sm_80 \
    -gencode arch=compute_80,code=compute_80 \
    -lcurand

In [13]:
!./random_generation

Generated random numbers (first 10):
h_output[0] = 0.195114
h_output[1] = 0.337967
h_output[2] = 0.433073
h_output[3] = 0.350870
h_output[4] = 0.618155
h_output[5] = 0.775106
h_output[6] = 0.623580
h_output[7] = 0.296973
h_output[8] = 0.197414
h_output[9] = 0.939234


## Example 5

### CUDA C++ (GPU)

In [14]:
%%writefile mcmc_sampler.cu
// File: mcmc_sampler.cu
#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include <curand_kernel.h>
#include <math.h> // For logf, expf, powf, sqrtf

// --- Device helper function to compute the log posterior ---
// A __device__ function can only be called from the device (e.g., from a kernel).
__device__ float log_posterior(float mu, const float *d_data, int N, float sigma2, float mu0, float tau2_0)
{
    // 1. Calculate log prior: log( N(mu | mu0, tau2_0) )
    float log_prior = -0.5f * powf(mu - mu0, 2) / tau2_0;

    // 2. Calculate log likelihood: log( N(data | mu, sigma2) )
    float log_likelihood = 0.0f;
    for (int i = 0; i < N; ++i)
    {
        log_likelihood += -0.5f * powf(d_data[i] - mu, 2) / sigma2;
    }

    return log_prior + log_likelihood;
}

// --- The MCMC Kernel ---
__global__ void mcmc_kernel(float *d_output, const float *d_data, curandState_t *states,
                            int N_data, int N_chains, int N_iters, int N_burn_in,
                            float sigma2, float mu0, float tau2_0, float prop_sigma)
{

    int idx = blockIdx.x * blockDim.x + threadIdx.x;

    if (idx < N_chains)
    {
        curandState_t local_rand_state = states[idx];
        float current_mu = mu0; // Start each chain at the prior mean

        // Main MCMC loop
        for (int i = 0; i < N_iters + N_burn_in; ++i)
        {
            // Propose a new mu using the Normal distribution from cuRAND
            float proposed_mu = current_mu + curand_normal(&local_rand_state) * prop_sigma;

            // Calculate log posterior for current and proposed mu
            float log_post_current = log_posterior(current_mu, d_data, N_data, sigma2, mu0, tau2_0);
            float log_post_proposed = log_posterior(proposed_mu, d_data, N_data, sigma2, mu0, tau2_0);

            // Acceptance check in log-space
            float log_alpha = fminf(0.0f, log_post_proposed - log_post_current);
            if (logf(curand_uniform(&local_rand_state)) < log_alpha)
            {
                current_mu = proposed_mu;
            }
        }

        // After burn-in and iterations, store the final sample of the chain
        d_output[idx] = current_mu;
        states[idx] = local_rand_state; // Save the updated random state
    }
}

// The setup_kernel from the previous section
__global__ void setup_kernel(curandState_t *states, unsigned long seed)
{
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    curand_init(seed, idx, 0, &states[idx]);
}

// --- The Main Host Function ---
int main()
{
    // --- 1. Problem Setup ---
    int N_data = 1000;        // Number of data points
    int N_chains = 1024 * 16; // Number of parallel MCMC chains to run (16,384)
    int N_iters = 2000;       // MCMC iterations per chain
    int N_burn_in = 500;      // Burn-in iterations to discard

    // True parameters for data generation
    float true_mu = 10.0f;
    float sigma2 = 4.0f;

    // Priors for mu ~ N(mu0, tau2_0)
    float mu0 = 0.0f;
    float tau2_0 = 100.0f;

    // MCMC proposal width
    float prop_sigma = 1.0f;

    // --- 2. Host-side Data Generation and Memory Allocation ---
    float *h_data = (float *)malloc(N_data * sizeof(float));
    float *h_output = (float *)malloc(N_chains * sizeof(float));

    // Generate synthetic data from the true model
    // Note: This host-side RNG is simple for demonstration.
    // In a real scenario, real data would be loaded.
    srand(time(NULL));
    for (int i = 0; i < N_data; ++i)
    {
        // A simple way to get a standard normal-like random number
        float u1 = rand() / (float)RAND_MAX;
        float u2 = rand() / (float)RAND_MAX;
        float rand_std_normal = sqrtf(-2.0f * logf(u1)) * cosf(2.0f * M_PI * u2);
        h_data[i] = true_mu + sqrtf(sigma2) * rand_std_normal;
    }

    // --- 3. Device-side Memory Allocation ---
    float *d_data, *d_output;
    curandState_t *d_states;
    cudaMalloc(&d_data, N_data * sizeof(float));
    cudaMalloc(&d_output, N_chains * sizeof(float));
    cudaMalloc(&d_states, N_chains * sizeof(curandState_t));

    // --- 4. Copy data and Setup Random States ---
    cudaMemcpy(d_data, h_data, N_data * sizeof(float), cudaMemcpyHostToDevice);

    int THREADS_PER_BLOCK = 256;
    int BLOCKS_PER_GRID = (N_chains + THREADS_PER_BLOCK - 1) / THREADS_PER_BLOCK;

    setup_kernel<<<BLOCKS_PER_GRID, THREADS_PER_BLOCK>>>(d_states, time(NULL));

    // --- 5. Launch the MCMC Kernel ---
    printf("Launching %d parallel MCMC chains...\n", N_chains);
    mcmc_kernel<<<BLOCKS_PER_GRID, THREADS_PER_BLOCK>>>(
        d_output, d_data, d_states, N_data, N_chains, N_iters, N_burn_in,
        sigma2, mu0, tau2_0, prop_sigma);
    cudaDeviceSynchronize();
    printf("MCMC simulation finished.\n");

    // --- 6. Copy results back and analyze ---
    cudaMemcpy(h_output, d_output, N_chains * sizeof(float), cudaMemcpyDeviceToHost);

    float posterior_mean = 0.0f;
    for (int i = 0; i < N_chains; ++i)
    {
        posterior_mean += h_output[i];
    }
    posterior_mean /= N_chains;

    printf("\n--- Analysis ---\n");
    printf("Posterior Mean of mu (from %d samples): %f\n", N_chains, posterior_mean);
    printf("True Mean of mu was: %f\n", true_mu);

    // --- 7. Cleanup ---
    cudaFree(d_data);
    cudaFree(d_output);
    cudaFree(d_states);
    free(h_data);
    free(h_output);

    return 0;
}


Writing mcmc_sampler.cu


In [15]:
!nvcc mcmc_sampler.cu -o mcmc_sampler \
    -gencode arch=compute_75,code=sm_75 \
    -gencode arch=compute_80,code=sm_80 \
    -gencode arch=compute_80,code=compute_80 \
    -lcurand

In [16]:
!time ./mcmc_sampler

Launching 16384 parallel MCMC chains...
MCMC simulation finished.

--- Analysis ---
Posterior Mean of mu (from 16384 samples): 9.948833
True Mean of mu was: 10.000000

real	0m1.567s
user	0m1.432s
sys	0m0.132s


### C++ (CPU)

In [17]:
%%writefile mcmc_sampler_cpu.cpp
// File: mcmc_sampler_cpu.cpp
#include <iostream>
#include <vector>
#include <random>
#include <cmath>   // For log, exp, pow, sqrt
#include <numeric> // For std::accumulate
#include <chrono>  // For seeding the random number generator

// --- Helper function to compute the log posterior on the CPU ---
// This is the direct equivalent of the __device__ function.
float log_posterior(float mu, const std::vector<float> &data, float sigma2, float mu0, float tau2_0)
{
    // 1. Calculate log prior: log( N(mu | mu0, tau2_0) )
    float log_prior = -0.5f * std::pow(mu - mu0, 2) / tau2_0;

    // 2. Calculate log likelihood: log( N(data | mu, sigma2) )
    float log_likelihood = 0.0f;
    for (float val : data)
    {
        log_likelihood += -0.5f * std::pow(val - mu, 2) / sigma2;
    }

    return log_prior + log_likelihood;
}

// --- The MCMC simulation function for the CPU ---
// This function replaces the CUDA kernel. It loops through each chain serially.
void run_mcmc_cpu(std::vector<float> &output, const std::vector<float> &data,
                  int N_chains, int N_iters, int N_burn_in,
                  float sigma2, float mu0, float tau2_0, float prop_sigma)
{

    // Setup a high-quality random number generator for the CPU
    unsigned seed = std::chrono::high_resolution_clock::now().time_since_epoch().count();
    std::mt19937 generator(seed);
    std::normal_distribution<float> prop_dist(0.0f, prop_sigma);
    std::uniform_real_distribution<float> uniform_dist(0.0f, 1.0f);

    // Loop over each chain (this was done in parallel on the GPU)
    for (int chain_idx = 0; chain_idx < N_chains; ++chain_idx)
    {
        float current_mu = mu0; // Start each chain at the prior mean

        // Main MCMC loop for this specific chain
        for (int i = 0; i < N_iters + N_burn_in; ++i)
        {
            // Propose a new mu using the normal distribution
            float proposed_mu = current_mu + prop_dist(generator);

            // Calculate log posterior for current and proposed mu
            float log_post_current = log_posterior(current_mu, data, sigma2, mu0, tau2_0);
            float log_post_proposed = log_posterior(proposed_mu, data, sigma2, mu0, tau2_0);

            // Acceptance check in log-space
            float log_alpha = std::min(0.0f, log_post_proposed - log_post_current);
            if (std::log(uniform_dist(generator)) < log_alpha)
            {
                current_mu = proposed_mu;
            }
        }

        // After burn-in and iterations, store the final sample of the chain
        output[chain_idx] = current_mu;
    }
}

// --- The Main Host Function ---
int main()
{
    // --- 1. Problem Setup ---
    int N_data = 1000;
    int N_chains = 1024 * 16; // Number of MCMC chains to run (16,384)
    int N_iters = 2000;
    int N_burn_in = 500;

    float true_mu = 10.0f;
    float sigma2 = 4.0f;

    float mu0 = 0.0f;
    float tau2_0 = 100.0f;

    float prop_sigma = 1.0f;

    // --- 2. Host-side Data Generation and Memory Allocation ---
    // Using std::vector for automatic memory management
    std::vector<float> h_data(N_data);
    std::vector<float> h_output(N_chains);

    // Generate synthetic data using C++'s <random> library
    unsigned seed = std::chrono::high_resolution_clock::now().time_since_epoch().count();
    std::mt19937 generator(seed);
    std::normal_distribution<float> data_dist(true_mu, std::sqrt(sigma2));

    for (int i = 0; i < N_data; ++i)
    {
        h_data[i] = data_dist(generator);
    }

    // --- 5. Launch the MCMC Simulation on the CPU ---
    printf("Launching %d serial MCMC chains on the CPU...\n", N_chains);
    run_mcmc_cpu(h_output, h_data, N_chains, N_iters, N_burn_in,
                 sigma2, mu0, tau2_0, prop_sigma);
    printf("MCMC simulation finished.\n");

    // --- 6. Analyze results ---
    // Use std::accumulate for a clean and efficient sum
    float posterior_mean = std::accumulate(h_output.begin(), h_output.end(), 0.0f);
    posterior_mean /= N_chains;

    printf("\n--- Analysis ---\n");
    printf("Posterior Mean of mu (from %d samples): %f\n", N_chains, posterior_mean);
    printf("True Mean of mu was: %f\n", true_mu);

    // --- 7. Cleanup ---
    // No manual free/delete needed thanks to std::vector!

    return 0;
}


Writing mcmc_sampler_cpu.cpp


In [18]:
!g++ -std=c++11 -O3 mcmc_sampler_cpu.cpp -o mcmc_sampler_cpu

In [19]:
!time ./mcmc_sampler_cpu

Launching 16384 serial MCMC chains on the CPU...
MCMC simulation finished.

--- Analysis ---
Posterior Mean of mu (from 16384 samples): 10.054132
True Mean of mu was: 10.000000

real	6m48.670s
user	6m48.557s
sys	0m0.050s
